In [1]:
import dataset
import time
import numpy as np
from PIL import Image
from matplotlib import pylab as plt
%matplotlib inline

from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Activation, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import History

Using TensorFlow backend.


In [2]:
types = [
        "Normal",
        "Fire",
        "Water",
        "Electric",
        "Grass",
        "Ice",
        "Fighting",
        "Poison",
        "Ground",
        "Flying",
        "Psychic",
        "Bug",
        "Rock",
        "Ghost",
        "Dragon",
        "Dark",
        "Steel",
        "Fairy"
]

In [3]:
images, (type1, type2) = dataset.load_data()

801/801 done


In [ ]:
type_dict = dataset.type_dict
type_labels = list(type_dict.keys())

img_rows = images.shape[1]
img_cols = images.shape[2]
img_channels = images.shape[3]
nb_classes = len(type_labels)  # 正解のパターン数

images = images.astype('float32') / 255.0  # 正規化

In [ ]:
type1_c = np_utils.to_categorical(type1, nb_classes)
type2_c = np_utils.to_categorical(type2, nb_classes)
types = type1_c + type2_c
types = np.where(types > 1., 1., types)

In [ ]:
# boder = 500

img_train = images[:500]
type_train = types[:500]
img_vali = images[500:750]
type_vali = types[500:750]
img_test = images[750:]
type_test = types[750:]

# 最後の可視化用
# type1_test = type1[750:]
# type2_test = type2[750:]

In [ ]:
input_layer = Input(shape=(img_rows, img_cols, img_channels))

x = Conv2D(32, kernel_size=(8,8), padding='same')(input_layer)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=(8,8), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Dropout(0.25)(x)

# x = Conv2D(64, kernel_size=3, padding='same')(x)
# x = Activation('relu')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D(pool_size=2)(x)
# x = Dropout(0.25)(x)

# x = Conv2D(128, kernel_size=3, padding='same')(x)
# x = Activation('relu')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D(pool_size=2)(x)
# x = Dropout(0.25)(x)

x = Flatten()(x)
# x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(nb_classes, activation="sigmoid")(x)

model = Model(inputs=input_layer, output=x)

In [ ]:
model.summary()

In [ ]:
def generator(X, Y, batch_size):
    indices = np.arange(X.shape[0])
    while True:
        img_cahce, label_cache = [], []
        np.random.shuffle(indices)
        for i in indices:
            img_cahce.append(X[i])
            label_cache.append(Y[i])
            
#             img = np.flip(X[i], axis=0)
#             img_cahce.append(img)
#             label_cache.append(Y[i])
            
            img = np.flip(X[i], axis=1)
            img_cahce.append(img)
            label_cache.append(Y[i])
            
#             img = np.flip(X[i], axis=0)
#             img = np.flip(img, axis=1)
#             img_cahce.append(img)
#             label_cache.append(Y[i])
            
            if len(img_cahce) == batch_size:
                X_batch = np.array(img_cahce)
                Y_batch = np.array(label_cache)
                img_cahce, label_cache = [], []
                yield X_batch, Y_batch

In [ ]:
model.compile(optimizer=Adam(lr=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = History()
batch_size = 32 # ４の倍数にしてくれ!!
model.fit_generator(generator(img_train, type_train, batch_size),
                   steps_per_epoch=len(img_train)*2//batch_size,
                   epochs=10,
                   verbose=1,
                   callbacks=[history],
                   validation_data=generator(img_vali, type_vali, batch_size),
                   validation_steps=len(img_test)//batch_size,)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].set_title('Training performance (Loss)')
ax[0].plot(history.epoch, history.history['loss'], label='loss')
ax[0].plot(history.epoch, history.history['val_loss'], label='val_loss')
ax[0].set(xlabel='Epoch', ylabel='Loss')
ax[0].legend()

ax[1].set_title('Training performance (Accuracy)')
ax[1].plot(history.epoch, history.history['acc'], label='acc')
ax[1].plot(history.epoch, history.history['val_acc'], label='val_acc')
ax[1].set(xlabel='Epoch', ylabel='Accuracy')
ax[1].legend(loc='best')

In [ ]:
def test_generator(X, batch_size):
    indices = np.arange(X.shape[0])
    while True:
        img_cahce = []
        np.random.shuffle(indices)
        for i in indices:
            img_cahce.append(X[i])
            if len(img_cahce) == batch_size:
                X_batch = np.array(img_cahce)
                img_cahce = []
                yield X_batch

In [ ]:
new_test = img_test

# 画像のラベルを推定する
batch_size = 1
pred = model.predict_generator(test_generator(img_test,batch_size),
                               steps=len(img_test)//batch_size,
                               verbose=1)

In [ ]:
pred[0]

In [ ]:
max(pred[0])

In [ ]:
types[9]